# 第8讲 机器学习入门

<font face="宋体" >郭峰（Email：guofengsfi@163.com)   
副教授、博士生导师  
上海财经大学公共经济与管理学院 </font> 

<font face="宋体" >本讲目录：  
8.1. 机器学习原理  
8.2. 线性回归、岭回归和Lasso回归  
8.3. K近邻算法  
8.4. 朴素贝叶斯算法  
8.5. 机器学习实操案例</font>  

## 8.5. 机器学习实操案例

<font face="宋体" >利用朴素贝叶斯根据人员姓名来作性别预测，原始数据来源于一个竞赛，训练样本：12万，成功率：约85%</font>  

### 文本数据特征工程：one-hot

In [ ]:
from IPython.display import Image
path='D:/python/郭峰Python讲义/数据与结果/08机器学习入门/'
Image(filename = path+'one_hot.png', width=500, height=260)

<font face="宋体" >one-hot表达文本是一个非常初级的方式，更高级的文本处理是正在蓬勃发展的自然语言处理(NLP)，超出了本课程的范围，本部分使用的仍然只是一个one-hot方法</font>  

### 导入第三方模块和数据

In [ ]:
#导入相关第三方库
import pandas as pd
import time
from collections import defaultdict
import joblib                #可能需要安装 pip install joblib
import datetime
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from numpy import *
from sklearn.model_selection import train_test_split
starttime = datetime.datetime.now()

#导入数据
path="D:/python/郭峰Python讲义/数据与结果/08机器学习入门/name_and_gender/"
data=pd.read_csv(path+'train.txt',encoding='utf8')  #一个竞赛网站12万样本
data['name']=data['name'].astype(str)
data['gender']=data['gender'].astype(int)
print(data.shape)
data.head(5)

In [ ]:
#如果数据量非常大，可以先将数据分出一部分，用于编写程序【仅用来演示，没有随机】
data_train=data.sample(10000) #如果样本非常大，可以先随机挑选一部分，用于编程演示
print("随机挑选一部分进行建模：",data_train.shape)

In [ ]:
#特征x是姓名用字，需要将x转换为一个数字化的向量
#所有姓名合并在一起，去重，构造一个姓名用字池向量
name_vec_total=list(data_train['name'])  
print(name_vec_total[0:20])

In [ ]:
name_vec_total=''.join(name_vec_total)
print(name_vec_total[0:20])
name_vec_total=list(''.join(name_vec_total))
print(name_vec_total[0:20])
print("语料库原始总字数：",len(name_vec_total))
print(237690/120000)

In [ ]:
print("不重复字样本量：",len(set(name_vec_total)))

In [ ]:
#去掉不常见的字
freq=defaultdict(int)
for w in name_vec_total:
    freq[w]+=1
name_vec_total=[w  for w in name_vec_total if freq[w]>5]
name_vec_total=list(set(name_vec_total)) #去重后再转换成列表
print("剔除稀缺字后不重复字样本量：",len(set(name_vec_total)))
print("不重复姓名用字举例:",name_vec_total[0:20])

In [ ]:
#姓名用字库保存一下
f=open(path+'name_vec_total.txt','w',encoding='utf8')
f.write(';'.join(name_vec_total))
f.close()

In [ ]:
#把具体某个姓名(如“建国”)的用字用上述姓名用字池向量来表示
def words2vec(inputSet): #inputSet是待定义姓名,这个函数基于上文得到的name_total
    returnVec = [0] * len(name_vec_total)    #获得所有单词等长的0列表
    for word in inputSet:
        if word in name_vec_total:
            returnVec[name_vec_total.index(word)] += 1   #对应单词位置加1
    return returnVec
#这个方式是在dataframe中计算
data_train['name_vec']=data_train['name'].apply(words2vec)

In [ ]:
#也可以先转换成list后再计算
name=list(data_train['name'])
print("姓名举例:",name[0:20])
name_vec=[words2vec(n) for n in name]   #特征x是用向量表示的姓名，这是一个嵌套列表，会占用内存超级多
print(name_vec[0])

In [ ]:
#相应y为gender，
gender_vec=list(data_train['gender'])
print(gender_vec[0:20])

In [ ]:
#训练集和验证集分割
x_train,x_test,y_train,y_test=train_test_split(name_vec,gender_vec,test_size=0.1)
print(len(x_train))
print(len(x_test))

In [ ]:
#朴素贝叶斯分类
#高斯，高斯结果一直不太好
bayes_g=GaussianNB()
bayes_g= bayes_g.fit(x_train,y_train)
y_pred=bayes_g.predict(x_test)
print("测试集预测：",y_pred[0:20])
print("Gaussian正确率 {:05.2f}%" .format(100*(1-(sum(array(y_pred)!=array(y_test))/len(y_test)))))


In [ ]:
#多项式
bayes_m=MultinomialNB()
bayes_m=bayes_m.fit(x_train,y_train)
y_pred=bayes_m.predict(x_test)
print("测试集预测：",y_pred[0:20])
print("MultinomialNB正确率 {:05.2f}%" .format(100*(1-(sum(array(y_pred)!=array(y_test))/len(y_test)))))


In [ ]:
#Bernoulli
bayes_b=BernoulliNB()
bayes_b=bayes_b.fit(x_train,y_train)
y_pred=bayes_b.predict(x_test)
print("验证集预测：",y_pred[0:20])
print("BernoulliNB验证集正确率 {:05.2f}%" .format(100*(1-(sum(array(y_pred)!=array(y_test))/len(y_test)))))


In [ ]:
#模型保存
joblib.dump(bayes_b,path+'bayes_bernoulli_nb'+'.model')  #模型的保存

In [ ]:
#test集的预测
path="D:/python/郭峰Python讲义/数据与结果/08机器学习入门/name_and_gender/"
data2=pd.read_csv(path+'test.txt',encoding='utf8')  #一个竞赛网站12万样本
data2['name']=data2['name'].astype(str)
print(data2.shape)
print(data2.head())

In [ ]:
name_new=list(data2['name']) 
print(name_new[0:20])

In [ ]:
x_test_new=[words2vec(n) for n in name_new]   
print(x_test_new[0])

In [ ]:
y_pred_new=bayes_b.predict(x_test_new)
print(y_pred_new[0:20])

In [ ]:
name_new=['峰']
print(name_new)
x_test_new=[words2vec(n) for n in name_new]   
#print(x_test_new[0])
y_pred_new=bayes_b.predict(x_test_new)
print(y_pred_new)

In [ ]:
#练习：可以尝试编写代码，将cssci数据集中的论文作者的性别推测出来

<font face="宋体" >其他常见的机器学习还包括：随机森林、支持向量机、神经网络等等，均超出了本课程的范围</font>  

<font face="微软雅黑" size=3>本讲结束</font>